In [ ]:
from pathlib import Path
import automol
import networkx as nx

from python import aml, orc, rdk, sql, hyq

data_dir = Path.cwd() / "data"
connection = sql.connect(data_dir)
sql.initialize_database(connection)


In [2]:
smiles = "C[CH]CC1CO1"

In [3]:
enumerated_graph = automol.smiles.graph(smiles)
geom = automol.graph.geometry(enumerated_graph)
automol.geom.display(geom, label=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
reactants = rdk.mol_from_smiles(smiles)

goat_pars = orc.ORCA_Parameters(
    xyz_in="guess.xyz",
    name_out="goat",
    multiplicity=2,
    functional="XTB",
    method_keywords="GOAT",
)

opt_pars = orc.ORCA_Parameters(
    xyz_in="goat.xyz",
    name_out="opt",
    multiplicity=2,
    functional="WB97X-3C",
    method_keywords="OPT",
)

freq_pars = orc.ORCA_Parameters(
    xyz_in="opt.xyz",
    name_out="freq",
    multiplicity=2,
    functional="M062X",
    basis="def2-TZVPP def2-TZVPP/c",
    method_keywords="DEFGRID3 TightSCF SlowConv Opt NumFreq",
    block_inputs="%geom\n MaxIter 500\nend",
    processors=16,
    max_memory=2000,
)

calc_pars = orc.ORCA_Parameters(
    xyz_in="freq.xyz",
    name_out="calc",
    multiplicity=2,
    functional="CCSD(T)-F12/RI",
    basis="cc-pVDZ-F12 cc-pVDZ-F12-CABS cc-pVTZ/c",
    processors=16,
    max_memory=4000,
)

scan_pars = orc.ORCA_Parameters(
    xyz_in="guess.xyz",
    name_out="scan",
    multiplicity=2,
    functional="XTB",
    method_keywords="ScanTS",
    processors=8,
    max_memory=1000,
    bash_commands="""
refined_file=$(ls scan.*.refined.xyz 2>/dev/null)
image_num=$(echo "$refined_file" | grep -oP "\d+")

prev=$(printf "%03d" $((10#$image_num - 1)))
cp scan.${prev}.xyz scan_prev.xyz

curr=$(printf "%03d" $((10#$image_num)))
cp scan.${curr}.xyz scan_curr.xyz

next=$(printf "%03d" $((10#$image_num + 1)))
cp scan.${next}.xyz scan_next.xyz

rm -f scan.[0-9][0-9][0-9].xyz
""",
)

optts_pars = orc.ORCA_Parameters(
    xyz_in="scan_curr.xyz",
    name_out="freq",
    multiplicity=2,
    functional="M062X",
    basis="def2-TZVPP def2-TZVPP/c",
    method_keywords="DEFGRID3 TightSCF SlowConv VeryTightOpt OptTS NumFreq",
    bash_commands="""
imag_mode=$(awk '
    $1 == "$vibrational_frequencies" {getline; n=$1; next}
    n && NF==2 && $2 < 0 {print $1; exit}
' "freq.hess")
orca_pltvib freq.log {{ imag_mode }}
""",
    processors=16,
    max_memory=2000,
)

task_graph = nx.DiGraph()
for reaction_smarts in [
    rdk.Reaction_Templates.PROTON_TRANSFER,
    rdk.Reaction_Templates.RING_OPENING,
]:
    product_sets = rdk.run_reaction(reactants, reaction_smarts, isomorphs=False)
    enumerated_graph = aml.process_rdkit_reaction(reactants, product_sets)
    to_submit = sql.enumerated_graph_into_database(enumerated_graph, data_dir, connection)

    for amchi, kind in to_submit.items():
        if kind == "stationary":
            # -- Global optimization algorithm
            goat_path = orc.orca_inputs(amchi, goat_pars, data_dir)
            task_graph.add_node(goat_path, pars=goat_pars)
            # -- Cheap optimization
            opt_path = orc.orca_inputs(amchi, opt_pars, data_dir)
            task_graph.add_node(opt_path, pars=opt_pars)
            task_graph.add_edge(goat_path, opt_path)
            # -- Optimization with frequency calculations
            freq_path = orc.orca_inputs(amchi, freq_pars, data_dir)
            task_graph.add_node(freq_path, pars=freq_pars)
            task_graph.add_edge(opt_path, freq_path)
            # -- Couple cluster
            calc_path = orc.orca_inputs(amchi, calc_pars, data_dir)
            task_graph.add_node(calc_path, pars=calc_pars)
            task_graph.add_edge(freq_path, calc_path)


        elif kind == "transition":
            data = enumerated_graph.nodes[amchi]
            # -- Geometry scan
            scan_pars.block_inputs = f"""%geom
 {data["scan"]} end
 TS_Mode {{B {data["active_atoms"]}}} end
 TS_Active_Atoms {{{data["active_atoms"]}}} end
end
"""
            scan_path = orc.orca_inputs(amchi, scan_pars, data_dir)
            task_graph.add_node(scan_path, pars=scan_pars)
            # -- TS optimization with frequency
            optts_pars.block_inputs = f"""%geom
 Hess_Internal
  {{B {data["active_atoms"]} C}}
  XYZ1 "scan_prev.xyz"
  XYZ2 "scan_next.xyz"
end
 MaxIter 500
end
"""
            optts_path = orc.orca_inputs(amchi, optts_pars, data_dir)
            task_graph.add_node(optts_path, pars=optts_pars)
            task_graph.add_edge(scan_path, optts_path)
            # -- Couple cluster
            calc_path = orc.orca_inputs(amchi, calc_pars, data_dir)
            task_graph.add_node(calc_path, pars=calc_pars)
            task_graph.add_edge(optts_path, calc_path)            

hyq.submit_tasks_orca(task_graph)

whitelist = [".allxyz", ".db", ".hess", ".inp", ".log", ".sh", ".xyz"]
to_delete = [path for path in data_dir.glob("**") if path.is_file() and path.suffix not in whitelist]

for file in to_delete:
    file.unlink()

sql.log_energies(connection, data_dir)
connection.close()

<>:51: SyntaxWarning: invalid escape sequence '\d'
<>:51: SyntaxWarning: invalid escape sequence '\d'
/tmp/tns97255-tmpdir-eoGfj8/ipykernel_2076563/73056861.py:51: SyntaxWarning: invalid escape sequence '\d'
  image_num=$(echo "$refined_file" | grep -oP "\d+")
2026-01-09T14:22:30Z INFO A trial allocation was submitted successfully. It was immediately canceled to avoid wasting resources.
2026-01-09T14:22:30Z INFO Allocation queue 1 successfully created
2026-01-09T14:22:30Z INFO A trial allocation was submitted successfully. It was immediately canceled to avoid wasting resources.
2026-01-09T14:22:30Z INFO Allocation queue 2 successfully created
2026-01-09T14:22:30Z INFO A trial allocation was submitted successfully. It was immediately canceled to avoid wasting resources.
2026-01-09T14:22:30Z INFO Allocation queue 3 successfully created
 96%|█████████▋| 55/57 [2:20:41<05:56, 178.34s/task, 0/1 job completed]